# Elementos Ópticos Difractivos

In [ ]:
from pyoptools.all import *

## Polinomios en pyOpTools

In [ ]:
%pylab inline
p2d=poly2d([0,0,0,1,0,1])
X,Y=meshgrid(linspace(-5,5,20),linspace(-5,5,20))

Z=p2d.eval(X,Y)
imshow(Z,extent=(-5,5,-5,5));colorbar()

## Ejemplo 1: Rejilla de difracción

In [ ]:
g=RPPMask(shape=Rectangular(size=(10,10)), phm=poly2d([0,2*pi*600,0,]),M=[1])#,0,-1,-2])

oc=Component(surflist=[(g, (0, 0, 0), (0, 0, 0)), 
                       ])
ccd=CCD(size=(1000,50))



os=System(complist=[(oc,(0,0,20),(0,0,0)),
                    (ccd,(0,0,120),(0,0,0)),
                    ],n=1)
r=[]
#Test using many wavelengths
for w in (.4,.45,.5,.53,.6,.65,.7):
    r.append(Ray(wavelength=w))

#Add the ray sources
os.ray_add(r)


os.propagate()

display(Plot3D(os,center=(0,0,60), size=(150,150),scale=2,rot=[(0,0,pi/2),(0,pi/2+0.1,0)]))
spot_diagram_c(ccd)


In [ ]:

n=1.
d=1/600.

for l in (.4e-3,.45e-3,.5e-3,.53e-3,.6e-3,.65e-3,.7e-3):
    th=arcsin(n*l/d)
    print("lambda = {:6.2E}  Intersección = {:4.3E}".format(l,100*tan(th)))

## Ejemplo 2: Corrigiendo aberraciones con un EOD

### Sistema aberrado

In [ ]:
L2=SphericalLens(radius=25,curvature_s1=1./100.,curvature_s2=-1./100,thickness=10,material=material.schott["BK7"])
L3=SphericalLens(radius=25,curvature_s1=1./100.,curvature_s2=-1./100,thickness=10,material=material.schott["BK7"])

C=CCD()

HOLO=CCD(size=(50,50))

S=System(complist=[(L2,(0,0,100),(0,0,0)),(HOLO,(0,0,110),(0,0,0)),(L3,(0,0,120),(0,0,0)),(C,(0,0,215),(0,0,0)) ],n=1)

R=point_source_p(origin=(0.,0.,0.),direction=(0.,0.,0),span=pi/14
                   ,num_rays=(50,50),wavelength=0.470, label="blue")

S.ray_add(R)
S.propagate()
Plot3D(S,center=(0,0,100),size=(250,100),scale=4,rot=[(0,pi/2,0),(pi/20,-pi/10,0)])

In [ ]:
spot_diagram(C)

In [ ]:
imshow(HOLO.get_optical_path_map())
colorbar()

In [ ]:
pol1,err1=HOLO.get_optical_path_map_lsq(order=15)

## Propagación de para atras

In [ ]:
L2=SphericalLens(radius=25,curvature_s1=1./100.,curvature_s2=-1./100,thickness=10,material=material.schott["BK7"])
L3=SphericalLens(radius=25,curvature_s1=1./100.,curvature_s2=-1./100,thickness=10,material=material.schott["BK7"])

C=CCD()

HOLO=CCD(size=(50,50))

S=System(complist=[(L2,(0,0,100),(0,0,0)),(HOLO,(0,0,110),(0,0,0)),(L3,(0,0,120),(0,0,0)),(C,(0,0,215),(0,0,0)) ],n=1)

R=point_source_p(origin=(0.,0.,215.),direction=(0.,pi,0.),span=pi/14
                   ,num_rays=(50,50),wavelength=0.470, label="blue")

S.ray_add(R)
S.propagate()
Plot3D(S,center=(0,0,100),size=(250,100),scale=4,rot=[(0,pi/2,0),(pi/20,-pi/10,0)])

In [ ]:
imshow(HOLO.get_optical_path_map())
colorbar()

In [ ]:
pol2,err2=HOLO.get_optical_path_map_lsq(order=15)

## Sistema corregido

In [ ]:
L2=SphericalLens(radius=25,curvature_s1=1./100.,curvature_s2=-1./100,thickness=10,material=material.schott["BK7"])
L3=SphericalLens(radius=25,curvature_s1=1./100.,curvature_s2=-1./100,thickness=10,material=material.schott["BK7"])

C=CCD()

HOLO=RPPMask(shape=Rectangular(size=(50,50)), phm=(2*pi/.470e-3)*(-pol1-pol2),M=[1])#,0,-1,-2])\
oc=Component(surflist=[(HOLO, (0, 0, 0), (0, 0, 0)), 
                       ])

S=System(complist=[(L2,(0,0,100),(0,0,0)),(oc,(0,0,110),(0,0,0)),(L3,(0,0,120),(0,0,0)),(C,(0,0,215),(0,0,0)) ],n=1)

R=point_source_p(origin=(0.,0.,0.),direction=(0.,0.,0),span=pi/18
                   ,num_rays=(10,10),wavelength=0.470, label="blue")

#R1=point_source_p(origin=(0.,0.,0.),direction=(0.,0.,0),span=pi/18
#                   ,num_rays=(10,10),wavelength=0.530)
S.ray_add(R)
#S.ray_add(R1)

S.propagate()
Plot3D(S,center=(0,0,100),size=(250,100),scale=4,rot=[(0,pi/2,0),(pi/20,-pi/10,0)])

In [ ]:
spot_diagram_c(C)
title("Diagrama de Manchas")

## Holograma

In [ ]:
holo=(2*pi/.470e-3)*(-pol1-pol2)

X,Y=meshgrid(linspace(-25,25,1000),linspace(-25,25,1000))

figsize(15,15)
imshow(holo.meval(X,Y)%(2*pi),extent=(-25,25,-25,25),cmap="gray")